# Ports and containers

Often, a container of class signals is required to cover complex bus connections, or simply create groups of signals for better overview. This is handled by the `@container` extensions. They can be seen as abstraction of VHDL records or SV structures.

A container is basically defined as follows:

In [1]:
from cyhdl import *

In [2]:
@container(CONTAINER_INTERFACE)
class Record:
    
    _inputs = ['a', 'en']
    _outputs = ['q']
    _other = []
    
    def __init__(self, n : int = 10, Signal = Signal):
        self.a = Signal(intbv()[8:])
        self.en = Signal(bool())
        self.q = Signal(intbv()[n:])
        self.n = n

The `CONTAINER_INTERFACE` argument denotes that a strict interface type definition should be used. In VHDL, this infers to a set of records with defined direction per record for each `_inputs`, `_outputs` and `_other`.

In fact, `CONTAINER_INTERFACE` is a Type Generator class, which functions as a type factory for each defined container type, or a derived container type of that sort. This implies, that strict type checking can be applied.

Further, we allow to pass a `context` parameter class to allow a better choice of that Signal to be used. This is useful as we see below.

Other types of Type Generators are present with other inference options.
In general, this new style is the most efficient to use.

To extend this Port, you can simply derive from it, for example by adding a `clk` and `reset` wire. Those are always inputs and must be added to the `_other` specification.

In [3]:
import myirl

class ExRecord(Record):
    _other  = ['clk', 'reset']
    
    def __init__(self, n, context = myirl):
        super().__init__(n, Signal = context.Signal)
        self.reset = context.ResetSignal(False, True)
        self.clk = context.ClkSignal()


In [4]:
xr10 = ExRecord(10)
xr10.get_children()

{'bulkc209_in': {`<class 'myirl.library.bulksignals.ExRecord_10_obj_module_in'>` | 'a', 'en'},
 'bulkc209_out': {`<class 'myirl.library.bulksignals.ExRecord_10_obj_module_out'>` | 'q'},
 'bulkc209_aux': {`<class 'myirl.library.bulksignals.ExRecord_10_obj_module_aux'>` | 'clk', 'reset'}}

### I/O handling

In complex port scenarios, in and output connections can become confusing. For example, when a hardware unit routes various port signals, in and outputs must be reversed and resolved correctly with respect to signal drivers and sources.

For the interface, a port reversal happens via a type hint as follows:

In [5]:
@block
def routing_unit(pin: ExRecord, pout: Record.reverse()):
    @always(pin.clk)
    def worker():
        pout.a.next = pin.a
        pout.en.next = pin.en
        pin.q.next = pout.q
        
    return worker

To display a VHDL instance of this `@block`:

In [6]:
p0 = ExRecord(6)
p1 = Record(6)
u = routing_unit(p0, p1)

files = u.elab(targets.VHDL)

 Writing 'routing_unit' to file /tmp/myirl_routing_unit_vs8yyivc/routing_unit.vhdl 


In [7]:
!cat {files[0]}

-- File generated from source:
--     /tmp/ipykernel_40328/3421992632.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.module_defs.all;
use work.txt_util.all;
use work.myirl_conversion.all;

entity routing_unit is
    port (
        pin_bulkc17c_in : in t_ExRecord_6_obj_module_in;
        pin_bulkc17c_out : out t_ExRecord_6_obj_module_out;
        pin_bulkc17c_aux : in t_ExRecord_6_obj_module_aux;
        pout_bulk19c7_in : out t_Record_6_obj_type_in;
        pout_bulk19c7_out : in t_Record_6_obj_type_out
    );
end entity routing_unit;

architecture cyriteHDL of routing_unit is
    -- Local type declarations
    -- Signal declarations
begin
    
worker:
    process(pin_bulkc17c_aux.clk)
    begin
        pout_bulk19c7_in.a <= pin_bulkc17c_in.a;
        pout_bulk19c7_in.en <= pin_bulkc17c_in.en;
        pin_bulkc17c_out.q <= pout_bulk19c7_out.q;
    

### Type checking

The `CONTAINER_INTERFACE` type generator has a beauty flaw when it comes to pythonic typing, as we can not make effective use type comparisons between instances of classes generated by it.

In [8]:
r12 = Record(12)
r12_x = ExRecord(12)
r9 = Record(9)

In [9]:
type(r12), type(r9)

(myirl.library.bulksignals.Record, myirl.library.bulksignals.Record)

However, their types are in fact not the same, as different port widths might be used in the member signals. Therefore this kind of type check can not be used internally. The interface type check has to rely on other mechanisms. However, dumping the children sheds some light on the situation:

In [10]:
r12.get_children()

{'bulkce16_in': {`<class 'myirl.library.bulksignals.Record_12_obj_type_in'>` | 'a', 'en'},
 'bulkce16_out': {`<class 'myirl.library.bulksignals.Record_12_obj_type_out'>` | 'q'}}

The Record entities functions as a container for a BULK type generated, unidirectional signal group which are created as classes using a certain signature. Therefore, the type check in the interface is performed on the children instead.
We simply demonstrate this on the simpler, unidirectional BULK type containers below.

## Bulk type containers

The unidirectional Bulk type containers do not require a input/output specification as the `CONTAINER_INTERFACE` variants. In VHDL, they translate into a single record type.

In [11]:
@container(CONTAINER_BULK)
class Port_bulk:
    def __init__(self, n, Signal = Signal):
        self.data = Signal(intbv()[n:])
        self.en = Signal(bool())
        
p4 = Port_bulk(4)
p8 = Port_bulk(8)
type(p4), type(p8)

(myirl.library.bulksignals.Port_bulk_4_obj_type,
 myirl.library.bulksignals.Port_bulk_8_obj_type)

Note we have created specific data types, so we can assert:

In [12]:
assert type(p4) != type(p8)
p = Port_bulk(4)
type(p) == type(p4)

True

This is better. We do get some usable type safety inside python, hence.
However note, that each differing instance of a Port_bulk alike class will create a separate data type.

## HDL translation

In VHDL, a `@container` class type infers into a type definition that is created in an ad-hoc work library. This happens automatically in most case and results in the creation of a `module_defs.vhdl` per design context. However, containers may register with a specific library that is imported. In this case, an external library reference is imported via a 'use' statement. In Verilog, this does not apply, as all Port class types are unrolled into children members.

In [13]:
files = u.elab(targets.Verilog)

 Writing 'routing_unit' to file /tmp/myirl_routing_unit_vs8yyivc/routing_unit.v 
DEBUG Fallback wire for pin_bulkc17c_aux_clk
DEBUG Fallback wire for pin_bulkc17c_aux_reset


In [14]:
!cat {files[0]}

// File generated from source:
//     /tmp/ipykernel_40328/3421992632.py
// (c) 2016-2022 section5.ch
// Modifications may be lost, edit the source file instead.

`timescale 1 ns / 1 ps
`include "aux.v"
// Architecture cyriteHDL
// Verilog: not creating library for bulkc17c_in
// Verilog: not creating library for bulkc17c_out
// Verilog: not creating library for bulkc17c_aux
// Verilog: not creating library for bulk19c7_in
// Verilog: not creating library for bulk19c7_out

module routing_unit
    (
        input wire [7:0] pin_bulkc17c_in_a,
        input wire  pin_bulkc17c_in_en,
        output reg [5:0] pin_bulkc17c_out_q,
        input wire  pin_bulkc17c_aux_clk,
        input wire  pin_bulkc17c_aux_reset,
        output reg [7:0] pout_bulk19c7_in_a,
        output reg  pout_bulk19c7_in_en,
        input wire [5:0] pout_bulk19c7_out_q
    );
    // Local type declarations
    // Signal declarations
    
    always @ (pin_bulkc17c_aux_clk) begin : WORKER
        pout_bulk19c7_in_a <=

## Macro extensions

In particular for hardware inference, specific port connection macros might be implemented. This is done in IRL notation for a container class, i.e. the `@hdlmacro` routines are explicit and never translated. For example, we extend the `ExRecord` class by a `@hdlmacro` function:

In [15]:
from myirl.kernel.sensitivity import _hdlmacro

class MExRecord(ExRecord):
    @hdlmacro
    def assign(self, other):
        assert isinstance(other.a, type(self.a))
        yield [
            other.q.set(self.q),
            self.en.set(other.en),
            self.a.set(other.a)
        ]

We can do runtime type checks *outside* the yield sequence, however note that in the CONTAINER case, they have to be done on their members, unlike the `CONTAINER_BULK` types.

In [16]:
@block
def routing_unit_lite(pin: MExRecord, pout: Record.reverse()):
    @always(pin.clk)
    def worker():
        pin.assign(pout)
        
    return worker

In [17]:
p0 = MExRecord(6)
p1 = Record(6)
u = routing_unit_lite(p0, p1)

files = u.elab(targets.VHDL)

 Writing 'routing_unit_lite' to file /tmp/myirl_routing_unit_lite_bwv4boyw/routing_unit_lite.vhdl 


When omitting the `@hdlmacro` decorator, you may get more control from the calling side of what is being generated.

Also note this container-specific `@hdlmacro` differs from the `@cyrite_factory.hdlmacro` construct as it always requires IRL notation.


The thumb rule number one is, that a `@hdlmacro` of a `@container` always (a priori) generates logic. It can not contain delay specifications or simulation constructs inside `yield` and it does not permit to be iterated by `yield from`.

### Abstraction #1: The method sequence decorator

A `@hdlmacro` is context agnostic (apart from class configuration) and always emits the same code, once configured. Also, it can in most cases not be used portably among different execution contexts.

Therefore, a new decorator is introduced that a priori allows to write function members in MyHDL style. It is silently context sensitive, when called from a CoSimulation sequence, it is run natively, when emit to simulation HDL, it is called as generator. 

In [18]:
class PortableRecord(MExRecord):
    @cyrite_method.sequence
    def assign_seq(self, other):
        self.assign(other).evaluate()
        
        yield delay(1)
    
    @cyrite_method.sequence
    def initialize(self):
        self.en.next = False
        self.a.next = 0
        yield delay(2)

Further sideband action like type checks can not be done inside this `@cyrite_method.sequence`, as it may be entirely translated to hardware. This again has to be buried in a `@hdlmacro` outside a yield, see [Out of band tricks](generators.ipynb#Out-of-band-tricks).

**Conclusion**: A `@hdlmacro` is always explicit, whereas a `@cyrite_method.sequence` is not!



## A test bench

For all the above, we'd like to see what it does. Again, according to [Simulation details](simulation.ipynb), we create a simulation setup that works for all targets.

Because we use a different type of Signal for various simulation backends, we pass the context specific `self.Signal` class to the container ininitalization.

**Note**: When using containers as simulation signals on test bench level, make sure to pass the entire container to the uut's interface, otherwise, signals may be left uninitialized. This will throw an error in Co-Simulation.

Also, unused signals may not get emitted to HDL back ends and throw errors. Thus, make sure to initialize port signal members completely when using ports from a top level.

In [19]:
from cyhdl import *

@block
def unit(
         clk : ClkSignal,
         i : Record,
         o : Record.reverse()):

    @always(clk.posedge)
    def not_ff():
        o.a.next = ~i.a

    return [ not_ff ]

class MyTest(cyrite_factory.Module):

    @cyrite_factory.testbench('ns')
    def tb(self):
        clk = self.ClkSignal(name='clk')

        # We can pass the 'name' parameter to make HDL output
        # and trace more readable
        p, q, r = \
            [ PortableRecord(8, context = self, name = name) for name in 'pqr' ]
        
        uut = unit(clk, p, r)

        @self.always(delay(2))
        def clkgen():
            clk.next = ~clk

        @self.sequence
        def main():
            yield from p.initialize()
            yield from q.initialize()
            
            p.en.next = False
            r.en.next = False
            q.q.next = 0 # Initialize unused output to avoid elimination
            p.a.next = 0x00
            yield delay(10)
            yield clk.negedge
            p.en.next = True
            p.a.next = 0xaa
            assert r.a == 0xff
            yield delay(1) # Update previous settings
            # Make sure to use 'yield from'!
            yield from q.assign_seq(p)
            # We now can expect values to be the same
            assert q.a == p.a
            print("r.a", r.a)

            # Expect updated result:
            yield clk.negedge
            assert r.a == 0x55
            
            raise StopSimulation
            
        return instances()

A common mistake is to forget that sequence must be called using `yield from` from a natively executed co-simulation top level `sequence`. When calling it like a function, it will work as emitted HDL simulation, but not within the co-routine main().

In [20]:
from yosys.simulator import CXXRTL
from cyrite.simulation.ghdl import GHDL

d = MyTest('all', CXXRTL)
t = d.tb()

t.run(2000, debug = True)

DEBUG: Dummysignal `s_7c87` : False -> False
DEBUG: Dummysignal `s_7c87` : False -> False
DEBUG: Dummysignal `s_19c1` : False -> False
DEBUG: Dummysignal `s_19c1` : False -> False
DEBUG: Dummysignal `s_9b38` : False -> False
DEBUG: Dummysignal `s_9b38` : False -> False
 Adding module with name `unit` 
 FINALIZE implementation `unit` of `unit` 
Compiling /tmp/myirl_all_oawji7aa/unit.pyx because it changed.
[1/1] Cythonizing /tmp/myirl_all_oawji7aa/unit.pyx
running build_ext
building 'runtime.unit' extension
creating build/temp.linux-x86_64-3.10/tmp/myirl_all_oawji7aa
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=unit -Iruntime -I/tmp/myirl_all_oawji7aa/ -I/usr/share/yosys/include/backends/cxxrtl/runtime -I/usr/local/include/python3.10 -c /tmp/myirl_all_oawji7aa/unit.cpp -o build/temp.linux-x86_64-3.10/tmp/myirl_all_oawji7aa/unit.o
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=un

Using '/tmp/myirl_all_oawji7aa/' for output
Cosimulation: a not connected to backend
Cosimulation: en not connected to backend
Cosimulation: q not connected to backend
STOP SIMULATION @20


**Important Note**: A simulation is a priori **not** simply portable among simulator backends. See [Simulator details](simulation.ipynb).